### Install dependencies

In [ ]:
%pip install llama-index-vector-stores-pinecone
%pip install pinecone-client>=3.0.0
%pip install "arize-phoenix[evals]" gcsfs nest-asyncio "llama-index>=0.10.3" "openinference-instrumentation-llama-index>=1.0.0" "llama-index-callbacks-arize-phoenix>=0.1.2"

### Initialize Phoenix for tracing

In [2]:
import phoenix as px
from llama_index.core import (
    set_global_handler,
    
)
session = px.launch_app()
set_global_handler("arize_phoenix")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/usr/local/anaconda3/envs/llm-apps/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


### Init Pinecone and set API keys

In [3]:
from pinecone import Pinecone, ServerlessSpec
from getpass import getpass
import sys
import os

os.environ["PINECONE_API_KEY"] = getpass("Please enter your Pinecone API key: ")
os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key: ")

api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=api_key)

In [4]:
pinecone_index = pc.Index("moonsync-index")

In [5]:
pinecone_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3222}},
 'total_vector_count': 3222}

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

### Basic RAG - No filtering

In [7]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("how does food affect women's mood during ovulation?")
print("Query response:")
print(response)

Query response:
Food can affect women's mood during ovulation by influencing the production of neurotransmitters like serotonin. Consuming healthy carbohydrates, whole grains, and foods rich in B-vitamins and other nutrients can support a calmer mood. On the other hand, quick sugar sources can lead to nutrient depletion and mood swings. Including adequate protein and fats in meals can help maintain stable blood sugar levels, which is important for managing mood during ovulation. Additionally, certain supplements and herbs can also play a role in reducing PMS symptoms and supporting mood regulation during the menstrual cycle.


### Setup AutoRetrieval - Metadata filtering

More information - https://docs.llamaindex.ai/en/stable/examples/vector_stores/pinecone_auto_retriever/

In [15]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo
from typing import List

vector_store_info = VectorStoreInfo(
    content_info="Women menstrual cycle, exercise, and diet",
    metadata_info=[
        MetadataInfo(
            name="categories",
            type="list[str]",
            description=(
                "Information category related to women, can be one or more of the following :" 
                "'diet/nutrition', 'general', 'fitness/wellness', 'mood/feeling'. "
             ),
        )
    ],
)
retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

### Create query engine using the retriever

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine

retriever_query_engine = RetrieverQueryEngine(
    retriever=retriever
)
response = retriever_query_engine.query("how does food affect women's mood during ovulation?")

In [17]:
print(response.response)

Hormonal changes during ovulation can lead to a decrease in cravings and appetite, making women feel more at home in their bodies. This can result in a positive impact on mood as they may feel more balanced and content.


### ChatEngine using our auto retrieval query engine

In [20]:
from llama_index.core.chat_engine import ContextChatEngine
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3000)
chat_engine = ContextChatEngine.from_defaults(
  retriever=retriever_query_engine,
  memory=memory,
  system_prompt= "You are MoonSync. You are asked to answers questions about women menstrual cycle, exercise, and diet. "
   "Always be emphatic and provide the best answer possible."
   "Examples below show the way you should approach the conversation." 
   "---------------------\n"
   "Example 1:\n"
   "Ashley: During PMS, my stomach gets upset easily, is there anything I can do to help?"
   "MoonSync: Hey Ashley! Sorry to hear, a lot of women struggle with this. I would recommend seeing a professional, but we can experiment together with common solutions so you’re armed with info when you see a specialist. Research suggests that dairy and refined wheats can inflame the gut during the follicular phase. Try avoiding them this week and let’s check in at the end to see if it helped. Depending on the outcome, happy to give you more recommendations."
   "---------------------\n"
   "Example 2:\n"
   "Ashely: I am preparing for a marathon and need to do some high intensity sprinting workouts as well as longer lower intensity runs. What’s the best way to plan this with my cycle?"
   "MoonSync: Hey Ashley, happy you asked! I’ll ask a few more details to get you on the best plan: when and how long is the marathon? How much are you running for your short and long trainings right now?"
)

In [21]:
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Assistant: Hey there! It sounds like you're going through a tough time, feeling overwhelmed and questioning your career choice. It's completely normal to have moments of doubt and uncertainty. Remember, it's okay not to have all the answers right now. It might be helpful to take a step back and reflect on what truly brings you joy and fulfillment. Have you considered talking to a career counselor or a therapist to explore your feelings and options further? Sometimes, having an outside perspective can provide valuable insights. Remember, it's never too late to make changes and pursue a path that aligns with your passions and values. Take care of yourself during this process, and remember that self-care is essential in times of uncertainty.

Assistant: When you're feeling overwhelmed and looking for a mood-boosting snack, it's essential to choose foods that can help stabilize your blood sugar and support your brain health. Here are som

In [22]:
chat_engine.reset()